In [2]:
import pinocchio as pin
import numpy as np
from __future__ import print_function
from numpy.linalg import norm, solve

In [9]:
# Create model and data

urdf_file_path = r'robot_description/Realman_Inspire_R/Realman_Inspire_R.singlearm.urdf'
robot = pin.buildModelFromUrdf(urdf_file_path)
data = robot.createData()

In [49]:

# x, y, z = 0, 0, 1
# roll, pitch, yaw = 0, 0, 0
# # Define the target position (XYZ) and orientation (RPY)
# target_position = np.array([x, y, z])  # Replace x, y, z with your values
# target_orientation_rpy = np.array([roll, pitch, yaw])  # Replace roll, pitch, yaw with your values


# # Define the target position and orientation (SE3 transform)
# [-0.17076383950188756, -0.23054557293653488, 1.0316324117593467, -0.1021298424195165, 0.608468522026588, 2.48366596965655]
# [0.4532258315011859, -0.5584450976457447, 0.35765426652505994, 0.7119130945540076, -1.0321086256166645, 2.2571554660469633]
target_position = np.array([0.4532258315011859, -0.5584450976457447, 0.35765426652505994])
target_orientation = pin.utils.rpyToMatrix( 0.7119130945540076, -1.0321086256166645, 2.2571554660469633)  # Replace with desired RPY angles
target_transform = pin.SE3(target_orientation, target_position)

In [43]:
# Set initial configuration (neutral position)
q_init = pin.neutral(robot)

JOINT_ID = 6
oMdes = pin.SE3(np.eye(3), np.array([1.0, 0.0, 1.0]))
oMdes

  R =
1 0 0
0 1 0
0 0 1
  p = 1 0 1

In [44]:
q = pin.neutral(robot)
q

array([0., 0., 0., 0., 0., 0., 0.])

In [45]:
# Perform inverse kinematics
eps = 1e-4
max_iter = 1000
DT = 1e-1
damp = 1e-12

In [46]:
end_effector_id = robot.getFrameId('R_middle_proximal')
end_effector_id

22

In [6]:
# pin.forwardKinematics(model, data, q)

In [51]:
for i in range(max_iter):
    # Compute forward kinematics
    pin.framesForwardKinematics(robot, data, q)
    
    # Compute the pose error (difference between current and target pose)
    current_transform = data.oMf[end_effector_id]
    error = pin.log6(current_transform.inverse() * target_transform).vector
    
    if np.linalg.norm(error) < eps:
        print("Converged after", i, "iterations")
        break
    
    # Compute the Jacobian for the end-effector frame
    J = pin.computeFrameJacobian(robot, data, q, end_effector_id, pin.ReferenceFrame.LOCAL)
    
    # Compute the change in configuration
    v = -damp * np.linalg.pinv(J) @ error
    q = pin.integrate(robot, q, v)

In [55]:
from pinocchio.utils import se3ToXYZQUAT

# Final configuration
print("Final joint configuration:", q)

# Verify by computing the final end-effector position
final_transform = pin.framesForwardKinematics(robot, data, q)[end_effector_id]
final_position, final_orientation = se3ToXYZQUAT(final_transform)

print("Final end-effector position:", final_position)
print("Final end-effector orientation (as quaternion):", final_orientation)

Final joint configuration: [-6.90053569e+03  1.83545554e+03 -1.43326147e+04 -3.51050374e+03
  3.58182034e+03  3.02657678e+03  1.29575118e-05]


KeyboardInterrupt: 

In [50]:
i = 0
while True:
    
    iMd = data.oMi[JOINT_ID].actInv(oMdes)
    err = pin.log(iMd).vector  # in joint frame
    if norm(err) < eps:
        success = True
        break
    if i >= max_iter:
        success = False
        break
    J = pin.computeJointJacobian(robot, data, q, JOINT_ID)  # in joint frame
    J = -np.dot(pin.Jlog6(iMd.inverse()), J)
    v = -J.T.dot(solve(J.dot(J.T) + damp * np.eye(6), err))
    q = pin.integrate(robot, q, v * DT)
    if not i % 10:
        print("%d: error = %s" % (i, err.T))
    i += 1

if success:
    print("Convergence achieved!")
else:
    print(
        "\nWarning: the iterative algorithm has not reached convergence to the desired precision"
    )

print("\nresult: %s" % q.flatten().tolist())
print("\nfinal error: %s" % err.T)

0: error = [ 7.85401753e-01 -7.85391769e-01  2.93500069e-01 -1.07633065e-07
 -1.08581565e-07 -1.57078363e+00]
10: error = [ 7.85401753e-01 -7.85391769e-01  2.93500069e-01 -1.07633065e-07
 -1.08581565e-07 -1.57078363e+00]
20: error = [ 7.85401753e-01 -7.85391769e-01  2.93500069e-01 -1.07633065e-07
 -1.08581565e-07 -1.57078363e+00]
30: error = [ 7.85401753e-01 -7.85391769e-01  2.93500069e-01 -1.07633065e-07
 -1.08581565e-07 -1.57078363e+00]
40: error = [ 7.85401753e-01 -7.85391769e-01  2.93500069e-01 -1.07633065e-07
 -1.08581565e-07 -1.57078363e+00]
50: error = [ 7.85401753e-01 -7.85391769e-01  2.93500069e-01 -1.07633065e-07
 -1.08581565e-07 -1.57078363e+00]
60: error = [ 7.85401753e-01 -7.85391769e-01  2.93500069e-01 -1.07633065e-07
 -1.08581565e-07 -1.57078363e+00]
70: error = [ 7.85401753e-01 -7.85391769e-01  2.93500069e-01 -1.07633065e-07
 -1.08581565e-07 -1.57078363e+00]
80: error = [ 7.85401753e-01 -7.85391769e-01  2.93500069e-01 -1.07633065e-07
 -1.08581565e-07 -1.57078363e+00]
90

0: error = [ 1.00000000e+00 -6.21510425e-23  2.93500000e-01  4.23516474e-22
  0.00000000e+00  0.00000000e+00]
10: error = [ 0.66836273 -0.05971668  0.49387237  0.04323127  0.00633189 -0.08447107]
20: error = [ 0.50034894 -0.90032731  1.12214353  0.55443365 -0.39567812 -1.54529059]
30: error = [-0.29325073  0.61856174  1.39896181 -1.31458706 -2.28447407 -0.74330618]
40: error = [ 1.28535095  0.33684004 -0.02047719 -0.26942102  1.21684237  0.47656536]
50: error = [ 0.76939219 -0.31175634  0.41268993 -0.83498435  0.35007096 -1.03315037]
60: error = [ 1.13116291 -0.10592561  0.16410771 -0.78380798  0.28940777 -0.76690316]
70: error = [ 1.42458907  0.17011997 -0.46422035 -1.2433568   1.61173142 -0.13952975]
80: error = [ 0.51158094 -0.08570834  0.61028012 -0.40789423 -0.29610659 -0.51134706]
90: error = [-0.21418769  0.8101648   1.39605085 -1.16227828 -1.78486942  0.14219453]
100: error = [ 0.34531732  0.15860907  0.74303263 -0.28608317 -0.82084455  0.08393997]
110: error = [ 1.25739977 -0.

In [8]:
q

array([-8.85266127e+03, -3.89395944e+02,  3.62358609e+04,  2.07471765e+01,
       -2.71505858e+04,  8.03597692e+02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00])